In [1]:
using ModelingToolkit, Plots, DifferentialEquations, Revise, ModelingToolkitStandardLibrary, Unitful
using Symbolics, Logging

@variables t
# Logging.disable_logging(Logging.@warn)
Logging.disable_logging(Logging.Warn)

LogLevel(1001)

## Connectors

In [31]:

Sym = Symbolics

cphe(x) = 5.192
cvhe(x) = 5.192 - 2.0769
@variables x
@register_symbolic cpfunc(x)
@register_symbolic kfunc(x)


# display(methods(cpfunc))
@connector function ThermoPin(; name, Pdef = 10.0, Tdef = 300, ṁdef = 0.0)
    @variables P(t) = Pdef              #[unit = u"bar"] 
    @variables ṁ(t) = ṁdef              #[unit = u"kg/s"] 
    @variables T(t)  = Tdef             #[unit = u"K"]
    @variables cp(t) = 5192
    @variables k(t)  = 1.667
    eq = [
        cp ~ cpfunc(T)
        k ~ kfunc(T)
    ]
    sts = [T, P, ṁ, cp, k]
    ODESystem(eq, t, sts, []; name = name, defaults = [P => Pdef, T => Tdef, ṁ => ṁdef] )
end

@connector function WorkPin(; name)
    sts = @variables Ẇ(t) = 0.0 #[output = true]
    ODESystem(Equation[], t, sts, []; name = name)
end

# convention = +Q = Qin
@connector function FixedHeatFlowPin(; name, Qin = 1e6)  #input in W
    sts = @variables Q̇(t)=Qin #[input = true]
    ps = @parameters Qin = Qin
    eqs=[
        Q̇ ~ Qin
    ]
    ODESystem(eqs,t, sts, ps; name = name, defaults = [Q̇ => Qin])
end

@connector function HeatTransferPin(; name)  #input in W
    @variables Q̇(t) 
    ODESystem(Equation[],t, [Q̇], []; name = name, defaults = [Q̇ => 0.0])
end


# tp = ThermoPin(name = :thermopin)
# hfp = HeatFlowPin(name = :hfp)
# wfp = WorkPin(name = :wp)

HeatTransferPin (generic function with 1 method)

#### Flow source

In [32]:
@component function FixedFlowSource(;name, mflow = 1.0, P = 10, T = 300)
    @named src = ThermoPin(Pdef = P, Tdef = T, ṁdef = mflow)
    ps = @parameters ṁ=mflow P=P T=T
    # 0 variables, 3 equations for pin
    eqs = [
        src.T ~ T
        src.P ~ P
        ṁ + src.ṁ ~ 0   #has to be negative
    ]
    ODESystem(eqs, t, [], ps; name = name, systems = [src], defaults = [P => P, T =>T, ṁ => mflow] )
end

@component function FlowSource(;name, mflow = 1.0, P = 10, T = 300)
    @named n = ThermoPin(Pdef = P, Tdef = T, ṁdef = mflow)
    @named p = ThermoPin(Pdef = P, Tdef = T, ṁdef = mflow)
    ps = @parameters ṁ=mflow P=P T=T
    # 0 variables, 3 equations for pin
    eqs = [
            0 ~ n.T - T
            0 ~ p.T - n.T 
            0 ~ P - n.P 
            0 ~ p.P - n.P
            0 ~ p.ṁ - ṁ
            0 ~ ṁ + n.ṁ]   #has to be negative]
    ODESystem(eqs, t, [], ps; name = name, systems = [n,p], defaults = [P => P, T =>T, ṁ => mflow] )
end

@component function ThermoHeatSource(; name, Qin = 1e6)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named h = FixedHeatFlowPin(Qin = Qin)
    #   0 variables 3 equations for pin
    eqs = [
        0 ~ p.ṁ + n.ṁ# p.ṁ ~ n.ṁ                               # conservation of mass
        n.T ~ p.T + h.Q̇/(p.ṁ*p.cp)
        n.P ~ p.P
    ]
    ODESystem(eqs,t,[],[]; name = name, systems = [p,n,h])
end

@component function ThermoHeatTransfer(; name)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named h = HeatTransferPin()
    st = @variables Q̇(t) C(t)
    eqs = [
        0 ~ p.ṁ + n.ṁ       # p.ṁ ~ n.ṁ                               # conservation of mass
        C ~ p.ṁ * p.cp          # duty
        0 ~ h.Q̇ - Q̇ 
        n.T ~ p.T + h.Q̇/(p.ṁ*p.cp)
        n.P ~ p.P
    ]
    ODESystem(eqs,t,[Q̇,C],[]; name = name, systems = [p,n,h])
end

@component function ThermoCompressor(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named w = WorkPin()

    ps = @parameters rp = rp η = η
    eqs = [
        0 ~ p.ṁ + n.ṁ #p.ṁ ~ n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T * ((1.0 - η - (rp^((p.k-1)/p.k))) / (-η))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    ODESystem(eqs,t,[],ps; name = name, systems = [p,n,w], defaults = [η => η, rp => rp])
end

@component function ThermoTurbine(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin(Pdef = 80, Tdef = 800)
    @named n = ThermoPin(Pdef = 80/3.5, Tdef = 500)
    @named w = WorkPin()
    ps = @parameters rp = rp η = η
    eqs = [
        0 ~ p.ṁ + n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T *(η + rp^((p.k-1)/p.k)- η*(rp^((p.k-1)/p.k))) / (rp^((p.k-1)/p.k))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    ODESystem(eqs,t,[],ps; name = name, systems =  [p,n,w],  defaults = [η => η, rp => rp])
end

@component function WorkSensor(;name)
    @named src = WorkPin()
    var = @variables Ẇ(t)
    eqs= [0 ~ Ẇ - src.Ẇ]
    ODESystem(eqs,t,var,[]; name = name, systems =  [src])
end

@component function PassiveElement(;name)
    @named p = ThermoPin()
    @named n = ThermoPin()
    sts = @variables Q̇(t) = 0.0 ΔP(t) ΔT(t) Δṁ(t)
    # 0 variables, 3 equations for pin

    eqs = [
        n.ṁ + p.ṁ ~ Δṁ    #has to be negative
        ΔT ~ p.T - n.T
        Q̇ ~ -(p.ṁ * p.cp * ΔT)
        ΔP ~ p.P - n.P
        ]
    ODESystem(eqs, t, [Q̇,ΔP,ΔT,Δṁ], []; name = name, systems = [p,n], defaults = [Q̇ => 0, ΔT =>0, ΔP => 0, Δṁ => 0])
end

@component function Regenerator(;name, ϵ = 0.95)
    @named A = ThermoHeatTransfer()
    @named B = ThermoHeatTransfer()
    ps = @parameters ϵ = ϵ
    @variables Q̇(t) 
    eqs = [
        Q̇ ~ ϵ * (A.p.T - B.p.T) * ((A.C < B.C) *  A.C + (A.C >= B.C) * B.C)
        0~A.Q̇ + B.Q̇
        A.Q̇ ~ Q̇
    ]
    ODESystem(eqs, t, [Q̇], ps; name = name, systems = [A,B], defaults = [ϵ =>0.95] )
end


Regenerator (generic function with 1 method)

In [33]:
function connect_pins(pins...)
    # mass balance
    eqs = [
        sum(pin -> pin.ṁ, pins) ~ 0.0, # sum(mdot) = 0
    ]

    # pressure balance
    for pin in pins[2:end]
        push!(eqs, pins[1].P ~ pin.P) # p_1 = p_2, p_1 = p_3,...
    end

    return eqs
end

"""
    basic_connect(pinA,pinB) 
    Stuff
"""
function basic_connect(pinA,pinB)   # creates thermoPin connections  P = P, T = T, mflow = mflow
    eqs = [
        sum(pin -> pin.ṁ, [pinA,pinB]) ~ 0.0, # sum(mdot) = 0
    ]
    push!(eqs, pinA.P ~ pinB.P)
    push!(eqs, pinA.T ~ pinB.T)
    return eqs
end


"""
    Dont use this for flowsource
"""
function series_connect(pins)
    # @assert length(pins)>2
    eqs =  basic_connect(pins[1].n,pins[2].p)   # n -> p

    if length(pins)>2
        for i = 2:length(pins)-1
            eqs = vcat(eqs,  basic_connect(pins[i].n,pins[i+1].p))
        end
    end
    return eqs
end

function hx_connect(hx, compAin, compAout, compBin, compBout)
    sideA = series_connect([compAin,hx.A,compAout])
    sideB = series_connect([compBin,hx.B,compBout])
    return vcat(sideA,sideB)
end0


hx_connect (generic function with 1 method)

In [38]:

flow    = FlowSource(name = :flow)
cmp     = ThermoCompressor(name = :cmp)
reg     = Regenerator(name = :regen)
heat    = ThermoHeatSource(name = :heat, Qin = 1e6)
trb     = ThermoTurbine(name = :trb)
psv     = PassiveElement(name = :psv)

eqs = series_connect([psv,flow,cmp])
eqs = vcat(eqs,series_connect([heat,trb]))

all_connections = vcat(eqs,hx_connect(reg,cmp,heat,trb,psv))
all_sys = [flow,cmp,reg,heat,trb,psv]


cphe(x) = 5192
khe(x) = 1.667
propDict = Dict(cpfunc => cphe, kfunc => khe)
all_connections

# flow.defaults
# cmp.defaults
# all_connections

Dict{Any, Any} with 2 entries:
  rp => rp
  η  => η

In [40]:
@named model = ODESystem(all_connections,t; systems = all_sys)
model = substitute(model,propDict)
odesys = structural_simplify(model)
parameters(odesys)
odesys.defaults

Dict{Any, Any} with 83 entries:
  cmp₊n₊T(t)      => 300
  regen₊B₊p₊k(t)  => 1.667
  trb₊p₊T(t)      => 800
  heat₊p₊P(t)     => 10.0
  cmp₊n₊cp(t)     => 5192
  psv₊p₊ṁ(t)      => 0.0
  trb₊p₊P(t)      => 80
  cmp₊w₊Ẇ(t)      => 0.0
  psv₊p₊P(t)      => 10.0
  flow₊p₊P(t)     => 10
  heat₊p₊k(t)     => 1.667
  regen₊A₊n₊cp(t) => 5192
  flow₊T          => 300
  heat₊p₊ṁ(t)     => 0.0
  regen₊A₊n₊T(t)  => 300
  regen₊B₊p₊T(t)  => 300
  trb₊n₊P(t)      => 22.8571
  psv₊Q̇(t)        => 0.0
  cmp₊p₊ṁ(t)      => 0.0
  ⋮               => ⋮

In [27]:
params = [
        flow.T    => 300
        flow.P    => 10.0
        flow.ṁ    => 10
        cmp.rp  => 3.5
        cmp.η   => 0.8
        reg.ϵ => 0.9
        heat.h.Qin => 1e6
        trb.rp    => 3.5
        trb.η     => 0.9
        ]

tspan = (0.0,10.0)
prob = ODEProblem(odesys,odesys.defaults,tspan,params)


ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: 1-element Vector{Float64}:
 300.0

In [30]:
# [println()]
# using Printf
for s in states(heat)
    @printf "\t %s = %.2d\n" s prob.f.observed(s, prob.u0, prob.p, 10.0)
end
# @show   prob.f.observed(trb.n.T, prob.u0, prob.p, 10.0)
# @show   prob.f.observed(reg.B.p.T, prob.u0, prob.p, 10.0)

ArgumentError: ArgumentError: p₊T(t) is neither an observed nor a state variable.

In [ ]:

@show win = prob.f.observed(cmp.w.Ẇ, prob.u0, prob.p, 1.0)
@show prob.f.observed(cmp.p.T, prob.u0, prob.p, 1.0)
@show prob.f.observed(cmp.n.T, prob.u0, prob.p, 1.0)
@show qTi = prob.f.observed(heat.p.T, prob.u0, prob.p, 1.0)
@show qTo = prob.f.observed(heat.n.T, prob.u0, prob.p, 1.0)
@show qin = prob.f.observed(heat.h.Q̇, prob.u0, prob.p, 1.0)
@show   twork = prob.f.observed(trb.w.Ẇ, prob.u0, prob.p, 1.0)
@show   prob.f.observed(trb.n.T, prob.u0, prob.p, 1.0)
@show prob.f.observed(reg.Q̇,prob.u0,prob.p,1.0)
# @show  qwst = prob.f.observed(psv.Q̇, prob.u0, prob.p, 9.0)
# @show qin + win 
# @show twork
# @show twork + qin + win



In [ ]:

mf = FlowSource(name = :mf);
cc = ThermoCompressor(name = :cc)
qq = ThermoHeat(name = :qq)
tt = ThermoTurbine(name = :tt)

con = [connect(mf.src,cc.p)
        connect(cc.n,qq.p)
        connect(qq.n,tt.p)]

        
@named connected_sys =  NonlinearSystem(con,[],[]; systems = [mf, cc, qq, tt]);
# updaded_eq = substitute(equations(expand_connections(connected_sys)))
@named newsys = NonlinearSystem(equations(expand_connections(connected_sys)),states(connected_sys),parameters(connected_sys))
simple_sys = alias_elimination(newsys)
parameters(simple_sys)
# states(simple_sys)
# showeq(simple_sys)